In [17]:
import numpy as np
import pandas as pd
import data as dt
import functions as fn
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [18]:
BTCUSDT15m = dt.BTCUSDT15m

In [19]:
def RSI(precios, n):
    delta = precios['Close'].diff()
    delta = delta[1:]

    sube, baja = delta.copy(),delta.copy()
    sube[sube<0] = 0
    baja[baja>0] = 0

    # Calculamos las medias de subida y de bajada
    media_sube = sube.rolling(n).mean()
    media_baja = baja.abs().rolling(n).mean()

    RS = media_sube/media_baja

    # Calculamos el índice RSI
    return 100 - (100/(1+RS))

    

In [20]:
n = 14
n2 = 17

BTCUSDT15m['RSI_1'] = RSI(BTCUSDT15m,n)
#BTCUSDT15m['RSI_2'] = RSI(BTCUSDT15m,n)

# Stoch RSI =  ( RSI - Lowest RSI )   /  ( Max RSI - Lowest RSI)

# RSI = Current RSI reading
# Lower RSI = Minimum RSI reading since the last 14 oscillations
# Max RSI = Maximum RSI reading since the last 14 oscillations

def Stoch_RSI(df,n,rsi_name):
    Stoch_RSI_t = []
    for i in range(len(df[rsi_name])-n):
        temp = df[rsi_name][i:i+n]
        L_RSI = np.min(temp)
        M_RSI = np.max(temp)
        RSI = temp.values[-1]

        Stoch_RSI_t.append(( RSI - L_RSI )   /  ( M_RSI - L_RSI))


    Stoch_RSI =  np.zeros(len(df))
    Stoch_RSI[n:] = np.array(Stoch_RSI_t)*100

    return Stoch_RSI

BTCUSDT15m['S_RSI_1'] = Stoch_RSI(BTCUSDT15m,n,'RSI_1')
BTCUSDT15m['D'] = BTCUSDT15m['S_RSI_1'].rolling(3).mean()

C:\Users\tonat\AppData\Local\Temp/ipykernel_61228/2824819379.py:21: RuntimeWarning:

invalid value encountered in double_scalars



In [22]:

#BTCUSDT15m.set_index("Open Time", inplace=True)
t1 = BTCUSDT15m['2021-03-18 19:00:00': '2021-03-21 00:00:00']

In [23]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=t1.index,
            open=t1["Open"],
            high=t1["High"],
            low=t1["Low"],
            close=t1["Close"],
            name="OHLC",
        ),
        go.Scatter(
            x=t1.index, y=t1['S_RSI_1'], mode="lines", name="RSI", yaxis="y2"
        ),
        go.Scatter(
            x=t1.index, y=t1['D'], mode="lines", name="D%", yaxis="y2"
        ),
    ],
 ).update_layout(
     yaxis_domain=[0.3, 1],
     yaxis2={"domain": [0, 0.20]},
     xaxis_rangeslider_visible=False,
     showlegend=False,
 )

fig_001

In [ ]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=t1.index,
            open=t1["Open"],
            high=t1["High"],
            low=t1["Low"],
            close=t1["Close"],
            name="OHLC",
        ),
        go.Scatter(
            x=t1.index, y=t1["RSI"], mode="lines", name="RSI", yaxis="y2"
        ),
        # go.Scatter(
        #     x=t1.index, y=t1["Stoch RSI"], mode="lines", name="RSI", yaxis="y2"
        # ),
    ],
).update_layout(
    yaxis_domain=[0.3, 1],
    yaxis2={"domain": [0, 0.20]},
    xaxis_rangeslider_visible=False,
    showlegend=False,
)
fig_001.show()